# Stormdays Tutorial

In [1]:
from stormvogel import pgc
from stormvogel.model import EmptyAction, ModelType
from stormvogel.show import show
from stormvogel.layout import Layout

init = "init"

def available_actions(s: str):
    if s == "init": # If we are in the initial state, we have a choice.
        return [pgc.Action(["study"]), pgc.Action(["don't study"])]
    else: # Otherwise, we don't have any choice, we are just a Markov chain. 
        return [pgc.Action([])]

def delta(s: str, a: pgc.Action):
    if "study" in a.labels:
        return [(1, "studied")]
    elif "don't study" in a.labels:
        return [(1, "didn't study")]
    elif s == "studied":
        return [(9/10, "pass test"), (1/10, "fail test")]
    elif s == "didn't study":
        return [(2/5, "pass test"), (3/5, "fail test")]
    else:
        return [(1, "end")]

def labels(s):
    return [s]

# For rewards, you have to provide a list. This enables multiple reward models if you use a non-singleton list.
def rewards(s: str, a: pgc.Action):
    if s == "pass test":
        return {"r1":100}
    if s == "didn't study":
        return {"r1":15}
    else:
        return {"r1":0}

pgc_study = pgc.build_pgc(
    delta=delta,
    initial_state_pgc=init,
    available_actions=available_actions,
    labels=labels,
    modeltype=ModelType.MDP,
    rewards=rewards
)
vis = show(pgc_study)

In [2]:
from stormvogel import pgc
from stormvogel.model import EmptyAction, ModelType
from stormvogel.show import show
from stormvogel.layout import Layout

init = "init"

def available_actions(s: str):
    if s == "init": # If we are in the initial state, we have a choice.
        return [pgc.Action(["study"]), pgc.Action(["don't study"])]
    elif s == "fail test":
        return [pgc.Action(["retry"]), pgc.Action(["give up"])]
    else: # Otherwise, we don't have any choice, we are just a Markov chain. 
        return [pgc.Action([])]

def delta(s: str, a: pgc.Action):
    if "study" in a.labels:
        return [(1, "studied")]
    elif "don't study" in a.labels:
        return [(1, "didn't study")]
    elif s == "studied":
        return [(9/10, "pass test"), (1/10, "fail test")]
    elif s == "didn't study":
        return [(2/5, "pass test"), (3/5, "fail test")]
    elif "retry" in a.labels:
        return [(1, "init")]
    elif "give up" in a.labels:
        return [(1, "end")]
    else:
        return [(1, "end")]

def labels(s):
    return [s]

# For rewards, you have to provide a list. This enables multiple reward models if you use a non-singleton list.
def rewards(s: str, a: pgc.Action):
    if s == "pass test":
        return {"r1":100}
    if s == "didn't study":
        return {"r1":15}
    else:
        return {"r1":0}

def rewards(s: str, a: pgc.Action):
    if s == "pass test":
        return [100]
    if s == "didn't study":
        return [15]
    else:
        return [0]

pgc_study = pgc.build_pgc(
    delta=delta,
    initial_state_pgc=init,
    available_actions=available_actions,
    labels=labels,
    modeltype=ModelType.MDP,
    rewards=rewards
)
vis = show(pgc_study)

ValueError: On input pair studied Action(labels=[]), the rewards function does not return a dictionary. Make sure to change it to the format {<rewardmodel>:<reward>,...}

In [10]:
from stormvogel.model_checking import model_checking

In [11]:
result = model_checking(pgc_study, "Rmax=? [F \"end\"]", scheduler=True)

In [12]:
vis = show(pgc_study, result=result)

In [13]:
import stormvogel.simulator
path = stormvogel.simulator.simulate_path(pgc_study, scheduler=result.scheduler, steps=100)
vis.highlight_path(path, "purple")